<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit_Kleinkinderkennung/blob/main/konvertierung_in_tf_lite_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tensorflow Object Detection API installieren

In [2]:
# Tensorflow installieren

!pip install tensorflow=="2.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 458.4 MB 30 kB/s 
     |████████████████████████████████| 14.9 MB 2.9 MB/s 
     |████████████████████████████████| 132 kB 57.5 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=5d1a40768b3ef55ec4353d641c0c20e4a18be71376f9934086d0b92297f23c29
  Stored in directory: /root/.cache/pip/wheels/f1/60/77/22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=a8b5d62ad2a8cdbee65ac29407a3698336cdb6c502a5404f36fc717da66632a9
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=72341 sha256=93019a9d980e94099c61a8860c6f7deca220f9d745dd3f822b4d84e941ceea49
  Stored in directory: /root/.cache/pip/wheels/5f/fd/

In [3]:
# Tensorflow Models klonen, wenn es noch nicht vorhanden ist

import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3632, done.
remote: Counting objects: 100% (3632/3632), done.
remote: Compressing objects: 100% (3019/3019), done.
remote: Total 3632 (delta 962), reused 1519 (delta 558), pack-reused 0
Receiving objects: 100% (3632/3632), 47.11 MiB | 19.11 MiB/s, done.
Resolving deltas: 100% (962/962), done.


In [4]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [6]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

I1207 20:23:45.721831 140178538256256 efficientnet_model.py:143] round_filter input=24 output=24
I1207 20:23:45.722162 140178538256256 efficientnet_model.py:143] round_filter input=40 output=40
I1207 20:23:47.104097 140178538256256 efficientnet_model.py:143] round_filter input=40 output=40
I1207 20:23:47.104421 140178538256256 efficientnet_model.py:143] round_filter input=80 output=80
I1207 20:23:49.708947 140178538256256 efficientnet_model.py:143] round_filter input=80 output=80
I1207 20:23:49.718919 140178538256256 efficientnet_model.py:143] round_filter input=112 output=112
I1207 20:23:51.932304 140178538256256 efficientnet_model.py:143] round_filter input=112 output=112
I1207 20:23:51.932709 140178538256256 efficientnet_model.py:143] round_filter input=192 output=192
I1207 20:23:52.929595 140178538256256 efficientnet_model.py:143] round_filter input=192 output=192
I1207 20:23:52.929921 140178538256256 efficientnet_model.py:143] round_filter input=320 output=320
I1207 20:23:53.39419

# Exportieren in TF-Lite Inference Graph (Frozen Model) --> nur SSD MobileNet verwendbar

In [ ]:
config_path = '/content/drive/MyDrive/training_effdetd1_ds2_8_20000_100/ssd_efficientdet_d1_640x640_coco17_tpu-8.config'
training_folder = '/content/drive/MyDrive/training_effdetd1_ds2_8_20000_100'

In [ ]:
output_directory = training_folder+'/inference_graph_tfLite'
%mkdir {output_directory}

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path {config_path} --trained_checkpoint_dir {training_folder} --output_directory {output_directory}

In [ ]:
# converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/My Drive/efficientdet_d0_coco17_tpu-32/saved_model/',signature_keys=['serving_default'])
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# tflite_model = converter.convert()

# with tf.io.gfile.GFile('model.tflite', 'wb') as f:
#   f.write(tflite_model)

In [ ]:
!pip install tflite_support_nightly

In [ ]:
%pwd
%cd {training_folder}
%pwd

In [ ]:
import tensorflow as tf

saved_model_dir = '/content/drive/MyDrive/training_effdetd1_ds2_8_20000_100/inference_graph/saved_model'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel director

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

# Save the model.
with open('effdetD1.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata
import flatbuffers
import os
from tensorflow_lite_support.metadata import metadata_schema_py_generated as _metadata_fb
from tensorflow_lite_support.metadata.python import metadata as _metadata
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_info
from tensorflow_lite_support.metadata.python.metadata_writers import metadata_writer
from tensorflow_lite_support.metadata.python.metadata_writers import writer_utils
 
ObjectDetectorWriter = object_detector.MetadataWriter
 
_MODEL_PATH = "/content/drive/MyDrive/training_effdetd1_ds2_8_20000_100/inference_graph/effdetD1.tflite"
_LABEL_FILE = "/content/drive/MyDrive/training_effdetd1_ds2_8_20000_100/inference_graph/labelmap.txt"
_SAVE_TO_PATH = "/content/drive/MyDrive/training_effdetd1_ds2_8_20000_100/inference_graph/effdetD1_meta.tflite"
 
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)
 

displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())
 
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "SSD_Detector"
model_meta.description = (
    "Identify which of a known set of objects might be present and provide "
    "information about their positions within the given image or a video "
    "stream.")
 
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.BoundingBoxProperties)
output_location_meta.content.contentProperties = (
    _metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
output_location_meta.content.contentProperties.type = (
    _metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (
    _metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2
 
output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "category"
output_class_meta.description = "The categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("labelmap.txt")
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
output_class_meta.associatedFiles = [label_file]
 
output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2
 
output_number_meta = _metadata_fb.TensorMetadataT()
output_number_meta.name = "number of detections"
output_number_meta.description = "The number of the detected boxes."
output_number_meta.content = _metadata_fb.ContentT()
output_number_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_number_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
 
group = _metadata_fb.TensorGroupT()
group.name = "detection result"
group.tensorNames = [
    output_location_meta.name, output_class_meta.name,
    output_score_meta.name
]
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [
    output_location_meta, output_class_meta, output_score_meta,
    output_number_meta
]
subgraph.outputTensorGroups = [group]
model_meta.subgraphMetadata = [subgraph]
 
b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()